In [3]:
#hackathon
#name kehali pansare

import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
# JSON data
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [23]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("restaurants.db")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()


conn.execute("DROP TABLE IF EXISTS restaurants;")
conn.executescript(sql_script)


restaurants = pd.read_sql_query("SELECT * FROM restaurants;", conn)
print(restaurants.head())


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [27]:
orders_users = orders.merge(users, on='user_id', how='left')


In [26]:
final_dataset = orders_users.merge(restaurants, on='restaurant_id', how='left')


In [29]:
# Save to CSV
final_dataset.to_csv("final_food_delivery_dataset.csv", index=False)

print("✅ final_food_delivery_dataset.csv created successfully!")


✅ final_food_delivery_dataset.csv created successfully!


In [35]:
#mcqs
#q1
#filter only gold members
gold_df = df[df['membership'] == 'Gold']

#calculate total amount per city

city_revenue = gold_df.groupby('city')['total_amount'].sum()

#find the city with highest revenue
max_city = city_revenue.idxmax()
max_revenue = city_revenue.max()

#results

print(f"The city with the highest revenue from Gold members is {max_city} with total revenue of {max_revenue}")



The city with the highest revenue from Gold members is Chennai with total revenue of 1080909.79


In [43]:
# q2
#Step 1 Average order per cuisine
avg_order_by_cuisine = final_dataset.groupby('cuisine')['total_amount'].mean()

# Step2 Find the top cuisine
top_cuisine = avg_order_by_cuisine.idxmax()
top_avg = avg_order_by_cuisine.max()

# result
print(f" The cuisine with the highest average order value is {top_cuisine} with an average of ₹{top_avg:,.2f} per order.")


 The cuisine with the highest average order value is Mexican with an average of ₹808.02 per order.


In [44]:
#q3
# Total order per user
total_per_user = df.groupby('user_id')['total_amount'].sum()

# Filter users with total > 1000
high_value_users = total_per_user[total_per_user > 1000]

# Count them
num_users = high_value_users.shape[0]

# Show result
print(f" Number of users who placed orders worth more than ₹1000 in total: {num_users}")


 Number of users who placed orders worth more than ₹1000 in total: 2544


In [49]:
# 4
#find rating range with highest revenue

# create the rating ranges using pd.cut on the rating column
final_dataset['rating_range'] = pd.cut(final_dataset['rating'], bins=[0, 2, 3, 4, 5], labels=['0-2', '2-3', '3-4', '4-5'])

#calculate total revenue per rating range
range_revenue = final_dataset.groupby('rating_range')['total_amount'].sum()

#find the range with the highest revenue
top_revenue_range = range_revenue.idxmax()

print(f"Highest revenue rating range: {top_revenue_range}")

Highest revenue rating range: 4-5


/tmp/ipython-input-1473395962.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  range_revenue = final_dataset.groupby('rating_range')['total_amount'].sum()


In [51]:
#q5
#filter gold members then group by city and cacluate avg order value ,then find the city with highest avg
gold_orders = df[df['membership']=='Gold']
top_city = gold_orders.groupby('city')['total_amount'].mean().idxmax()
print(f"Among Gold members, {top_city} has the highest average order value.")


Among Gold members, Chennai has the highest average order value.


In [54]:
#q6
#count distict restaurants per cuisine , calculate total revenue per cuisine ,combine into 1 df
top_cuisine = final_dataset.groupby('cuisine').agg(num_restaurants=('restaurant_id','nunique'),
                                         total_revenue=('total_amount','sum')) \
                .sort_values(['num_restaurants','total_revenue'], ascending=[True,False]).head(1)

print(top_cuisine)


         num_restaurants  total_revenue
cuisine                                
Chinese              120     1930504.65


In [55]:
#7
# count gold member orders, divide by total orders, convert to percentage,round it, and print.
gold_percentage = round((len(df[df['membership']=='Gold']) / len(df)) * 100)
print(f"💎 Gold members placed {gold_percentage}% of total orders.")


💎 Gold members placed 50% of total orders.


In [59]:
#q8
#group by restaurant ,calculate total orders nd avg value ,
# keep restaurants with <20 orders , pick the one with highest average nd print it.
top_restaurant = final_dataset.groupby('restaurant_name_y').agg(
    total_orders=('order_id','count'),
    avg_order_value=('total_amount','mean')
)
top_restaurant = top_restaurant[top_restaurant['total_orders'] < 20]
best = top_restaurant['avg_order_value'].idxmax()
print(f" Restaurant with highest average order value (<20 orders): {best}")


 Restaurant with highest average order value (<20 orders): Restaurant_294


In [65]:
#q9
#sum revenue for each city-cuisine combination, pick the pair with the most revenue, and print it.

combo_revenue = final_dataset.groupby(['membership','cuisine'])['total_amount'].sum()
top_combo = combo_revenue.idxmax()
top_revenue = combo_revenue.max()
print(f" The combination generating the highest revenue is {top_combo[0]} city with {top_combo[1]} cuisine, totaling ₹{top_revenue:,.2f}.")




 The combination generating the highest revenue is Regular city with Mexican cuisine, totaling ₹1,072,943.30.


In [68]:
#10
#find which quarter (Q1-Q4) has the most money by summing order amounts per quarter
df['quarter'] = pd.to_datetime(df['order_date']).dt.quarter
top_quarter = df.groupby('quarter')['total_amount'].sum().idxmax()
print(f" Highest revenue quarter: Q{top_quarter}")


 Highest revenue quarter: Q3


/tmp/ipython-input-1750428397.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['quarter'] = pd.to_datetime(df['order_date']).dt.quarter


In [74]:
#q11
#filter gold members and count orders then print it
print(f"total orders by gold members: {len(df[df['membership']=='Gold'])}")


total orders by gold members: 4987


In [71]:
#q12
#filter order frm hyderabad then sum the revenue nd print
print(f"hyderabad revenue: {round(df[df['city']=='Hyderabad']['total_amount'].sum())}")


Hyderabad revenue: 1889367


In [73]:
#q13
#count the number of unique user ids in the dataset ,each represents a user who placed at least one order
print(f" total distinct users: {df['user_id'].nunique()}")


 total distinct users: 2883


In [76]:
#q14
#select only Gold member orders then calculate the mean of total amount,round to 2 decimals and print.
print(f"average order value for Gold members: {round(df[df['membership']=='Gold']['total_amount'].mean(), 2)}")


average order value for Gold members: 797.15


In [79]:
#q15
# Use final_dataset which contains the 'rating' column from the restaurants data
high_rated_orders = final_dataset[final_dataset['rating'] >= 4.5]
print(f"orders with rating ≥ 4.5: {len(high_rated_orders)}")

orders with rating ≥ 4.5: 3374


In [80]:
#q16
gold_orders = df[df['membership']=='Gold']
top_city = gold_orders.groupby('city')['total_amount'].sum().idxmax()
print(f"orders by Gold members in top revenue city ({top_city}): {len(gold_orders[gold_orders['city']==top_city])}")



orders by Gold members in top revenue city (Chennai): 1337
